In [8]:
import os
from typing import Callable, Tuple

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
from common import ModelManager

class LeNet5(nn.Module):
    def __init__(self) -> None:
        super(LeNet5, self).__init__()
        self.feature_extraction: nn.Sequential = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh(),
        )

        self.classifier: nn.Sequential = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=10),
        )
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        x = self.feature_extraction(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs


In [9]:
if __name__ == "__main__":
    RANDOM_SEED = 42
    LEARNING_RATE = 0.001
    BATCH_SIZE = 32
    N_EPOCHS = 15

    IMG_SIZE = (32, 32)

    data_path: str = os.path.join("resources", "datasets", "mnist")
    os.makedirs(data_path, exist_ok=True)
    
    data_transform: Callable = transforms.Compose([
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor(),
    ])

    train_data: datasets.MNIST = datasets.MNIST(
        root=data_path,
        train=True,
        transform=data_transform,
        download=True,
    )
    test_data: datasets.MNIST = datasets.MNIST(
        root=data_path,
        train=False,
        transform=data_transform,
        download=True,
    )

    train_data_loader: DataLoader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_data_loader: DataLoader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

    model_manager: ModelManager = ModelManager(
        model=LeNet5(),
        loss_fn=nn.CrossEntropyLoss(),
        optimizer=optim.Adam,
        hyperparameters=dict(
            lr=LEARNING_RATE,
        )
    )

    model_manager.run_train_loop(
        train_data_loader=train_data_loader,
        test_data_loader=test_data_loader,
        epochs=N_EPOCHS,
    )

    

    



Epoch 1
-------------------------------
Run training using cpu
loss: 2.301534 [    0/60000]
loss: 1.568970 [ 3200/60000]
loss: 1.694892 [ 6400/60000]
loss: 1.607045 [ 9600/60000]
loss: 1.488115 [12800/60000]
loss: 1.577436 [16000/60000]
loss: 1.559747 [19200/60000]
loss: 1.584497 [22400/60000]
loss: 1.470019 [25600/60000]
loss: 1.544739 [28800/60000]
loss: 1.489646 [32000/60000]
loss: 1.463840 [35200/60000]
loss: 1.506276 [38400/60000]
loss: 1.601617 [41600/60000]
loss: 1.529379 [44800/60000]
loss: 1.524057 [48000/60000]
loss: 1.468177 [51200/60000]
loss: 1.468789 [54400/60000]


KeyboardInterrupt: 